In [1]:
import sys
sys.path.append('/cluster/sj1/bb_opt/scripts')

In [2]:
%load_ext autoreload
%autoreload 2

import torch
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collect_stats as cs

%matplotlib inline

In [3]:
import collect_stats as cs

In [5]:
exp_folder = '/cluster/sj1/bb_opt/experiments'
test_filepath = '/cluster/sj1/bb_opt/data/tatsu_pbm/test_files'
filedir = '/cluster/sj1/bb_opt/data/paper_data2'
num_tfs_to_test = 38
batches = [10, 20, 100, 200]
batches = [10]
num_acks = 30

In [6]:
with open(test_filepath, 'r') as f:
    filenames = [k.strip() for k in f.readlines()][:num_tfs_to_test]

In [7]:
def sigmoid(x, exp=np.exp):
  return 1.0 / (1.0 + exp(-x))

In [8]:
filenames = [
    'HESX1_E149K_R1_8mers.txt',
    'EGR2_D383Y_R1_8mers.txt',
    'BCL6_H676Y_R1_8mers.txt',
    'ARX_L343Q_R1_8mers.txt',
    'CRX_E80A_R1_8mers.txt',
    'ESX1_K193R_R1_8mers.txt',
    'FOXC1_F112S_R1_8mers.txt',
    'GFI1B_A204T_R1_8mers.txt',
    'GFI1_L400F_R1_8mers.txt',
    'HOXC4_N178S_R1_8mers.txt',
    'HOXD13_I322L_R1_8mers.txt',
    'ISX_R83Q_R1_8mers.txt',
    'KLF11_R402Q_R1_8mers.txt',
    'KLF1_E325K_R1_8mers.txt',
    'MSX2_P148H_R1_8mers.txt',
    'NKX2-5_K183E_R1_8mers.txt',
    'NR1H4_C144R_R1_8mers.txt',
    'NR2E3_G56R_R1_8mers.txt',
    'OVOL2_D228E_R1_8mers.txt',
    'PAX3_G48R_R1_8mers.txt',
    'PAX4_R183C_R1_8mers.txt',
    'PAX6_A79E_R1_8mers.txt',
    'PAX7_P112L_R1_8mers.txt',
    'PBX4_R215Q_R1_8mers.txt',
    'PHOX2B_Q143R_R1_8mers.txt',
    'PITX2_L100Q_R1_8mers.txt',
    'POU3F4_A237G_R1_8mers.txt',
    'POU4F3_K277R_R1_8mers.txt',
    'POU6F2_E639K_R1_8mers.txt',
    'PROP1_R112Q_R1_8mers.txt',
    'SIX6_H141N_R1_8mers.txt',
    'SNAI2_D119E_R1_8mers.txt',
    'VAX2_L139M_R1_8mers.txt',
    'VENTX_E101K_R1_8mers.txt',
    'VSX1_G160D_R1_8mers.txt',
    'WT1_F392L_R1_8mers.txt',
    'ZNF200_H322Y_R1_8mers.txt',
    'ZNF655_E327G_R1_8mers.txt',
]

In [9]:
tf_max = {}
tf_labels = {}
for tf in filenames:
    labels = np.load(filedir + "/" + tf + "/labels.npy")
    labels = np.log(labels)
    tf_max[tf] = float(labels.max())
    tf_labels[tf] = labels

In [10]:
arrs = {}

In [11]:
import collect_stats as cs

In [76]:
to_read = {
    #'ensemble14/o_none_ucb_': [None, 'normal'],
    #'ensemble14/o_none_ucb_maxinvar_g000510204080_': [None, 'maxinvar_g80'],
    #'ensemble14/o_none_ucb_maxvar_g000510204080_': [None, 'maxvar_g80'],
    #'ensemble14/o_none_ucb_adve_': [None, 'adve'],
    'ensemble14/o_none_ucb_nc_g000510204080_': [None, 'nc'],
    #'ensemble14/o_none_ucb_mincov_g000510204080_': [None, 'mincorr'],
}

In [77]:
for k in to_read:
    arrs[k] = to_read[k]
for experiment in to_read:
    print('reading', experiment)
    arrs[experiment][0] = cs.get_data(exp_folder, experiment, batches, num_samples=20, mode="bayes_opt")
    #cs.get_data(exp_folder, experiment, batches, num_samples=20, mode="bayes_opt")

reading ensemble14/o_none_ucb_nc_g000510204080_
reading sample 1
reading sample 2
reading sample 3
reading sample 4
reading sample 5
reading sample 6
reading sample 7
reading sample 8
reading sample 9
reading sample 10
reading sample 11
reading sample 12
reading sample 13
reading sample 14
reading sample 15
reading sample 16
reading sample 17
reading sample 18
reading sample 19
reading sample 20


In [78]:
to_eval = [
    'ensemble14/o_none_ucb_',
    'ensemble14/o_none_ucb_maxinvar_g000510204080_',
    'ensemble14/o_none_ucb_maxvar_g000510204080_',
    'ensemble14/o_none_ucb_adve_',
    'ensemble14/o_none_ucb_nc_g000510204080_',
    #'ensemble14/o_none_ucb_mincov_g000510204080_',
]

In [18]:
idx_data_extractor = lambda x, filename : x['idx_frac'][0].item()

In [19]:
ack_rel_opt_val_data_extractor = lambda x, filename : x['ack_rel_opt_value']

In [20]:
train_rmse_fn = lambda x, filename : x['logging'][1]['train']['std']

In [21]:
def ir_regret_fn(x, filename): 
    temp = x['ir_batch_cur_idx'].numpy().tolist()
    temp2 = set(x['ack_idx'].numpy().tolist())
    temp3 = None
    for k in temp[::-1]:
        if k not in temp2:
            break
    return np.exp(max(tf_labels[filename][k], tf_labels[filename][x['ack_idx']].max()))

def ack_regret_fn(x, filename): 
    return np.exp(max(tf_labels[filename][x['ack_idx']].max(), tf_labels[filename][x['idx_at_each_iter'][0]].max()))
    #return np.exp(tf_labels[filename][x['ack_idx']].max())



In [79]:
cs.prop_test(10, 
             filenames,
             idx_data_extractor,
             #ack_rel_opt_val_data_extractor, 
             arrs, 
             [to_eval[4], to_eval[2]],
             29,
             pval_threshold=1,
             paired_test=True,
            )

comparing nc maxvar_g80
HESX1_E149K_R1_8mers.txt 0.06690432258177074 0.39588414803147315 0.4222561001777649 1 	 (std: 0.08566 0.07139, #n: 20 20)
EGR2_D383Y_R1_8mers.txt 0.14187430931777045 0.12012195060960948 0.14298780364915728 1 	 (std: 0.07681 0.07557, #n: 20 20)
BCL6_H676Y_R1_8mers.txt 0.18130763764497881 0.40015244036912917 0.3916158556938171 0 	 (std: 0.02151 0.02680, #n: 20 20)
ARX_L343Q_R1_8mers.txt 0.4266577473903904 0.1797256089746952 0.18490853793919088 1 	 (std: 0.02611 0.02936, #n: 20 20)
CRX_E80A_R1_8mers.txt 0.8204319271908568 0.16265243850648403 0.16021341532468797 0 	 (std: 0.03451 0.04136, #n: 20 20)
ESX1_K193R_R1_8mers.txt 0.464796114867575 0.42591463178396227 0.4391768276691437 1 	 (std: 0.09094 0.05902, #n: 20 20)
FOXC1_F112S_R1_8mers.txt 0.26178534940563736 0.208384145796299 0.2041158527135849 0 	 (std: 0.02241 0.02463, #n: 20 20)
GFI1B_A204T_R1_8mers.txt 0.3081142573682527 0.09146341532468796 0.08262195196002722 0 	 (std: 0.03405 0.04144, #n: 20 20)
GFI1_L400F_R

In [80]:
cs.prop_test(10,
             filenames,
             ack_regret_fn,
             #ack_rel_opt_val_data_extractor,
             arrs,
             [to_eval[4], to_eval[2]],
             29,
             pval_threshold=1,
             paired_test=True,
            )

comparing nc maxvar_g80
HESX1_E149K_R1_8mers.txt 0.10571259403231917 18972.363 20547.2 1 	 (std: 3846.60522 3607.98804, #n: 20 20)
EGR2_D383Y_R1_8mers.txt 0.5517484417292368 66774.23 71291.375 1 	 (std: 24099.80078 28775.30078, #n: 20 20)
BCL6_H676Y_R1_8mers.txt 0.18641143545847969 10468.777 10375.498 0 	 (std: 135.53026 248.74219, #n: 20 20)
ARX_L343Q_R1_8mers.txt 0.3732390185148047 82187.96 81978.33 0 	 (std: 827.33185 629.36133, #n: 20 20)
CRX_E80A_R1_8mers.txt 0.22070353434595305 330505.62 323630.75 0 	 (std: 21662.79102 20437.97461, #n: 20 20)
ESX1_K193R_R1_8mers.txt 0.0359742806104311 24964.824 26922.6 1 	 (std: 3517.72021 2005.68713, #n: 20 20)
FOXC1_F112S_R1_8mers.txt 0.9125714024358829 84778.77 84773.89 0 	 (std: 204.05176 214.90680, #n: 20 20)
GFI1B_A204T_R1_8mers.txt 0.46266884464337565 144654.34 142769.66 0 	 (std: 8650.21680 9239.40820, #n: 20 20)
GFI1_L400F_R1_8mers.txt 0.23297753143389824 78964.95 83128.266 1 	 (std: 12648.97168 13194.69922, #n: 20 20)
HOXC4_N178S_R1_8me

In [81]:
cs.prop_test(10, 
             filenames, 
             ir_regret_fn, 
             arrs, 
             [to_eval[4], to_eval[2]], 
             29,
             pval_threshold=1,
             paired_test=True,
            )

comparing nc maxvar_g80
HESX1_E149K_R1_8mers.txt 0.10571259403231917 18972.363 20547.2 1 	 (std: 3846.60522 3607.98804, #n: 20 20)
EGR2_D383Y_R1_8mers.txt 0.6053737428117263 67490.98 71291.375 1 	 (std: 23536.30273 28775.30078, #n: 20 20)
BCL6_H676Y_R1_8mers.txt 0.18641143545847969 10468.777 10375.498 0 	 (std: 135.53026 248.74219, #n: 20 20)
ARX_L343Q_R1_8mers.txt 0.3732390185148047 82187.96 81978.33 0 	 (std: 827.33185 629.36133, #n: 20 20)
CRX_E80A_R1_8mers.txt 0.2963538981548161 330505.62 324729.44 0 	 (std: 21662.79102 20225.24219, #n: 20 20)
ESX1_K193R_R1_8mers.txt 0.0359742806104311 24964.824 26922.6 1 	 (std: 3517.72021 2005.68713, #n: 20 20)
FOXC1_F112S_R1_8mers.txt 0.9125714024358829 84778.77 84773.89 0 	 (std: 204.05176 214.90680, #n: 20 20)
GFI1B_A204T_R1_8mers.txt 0.46266884464337565 144654.34 142769.66 0 	 (std: 8650.21680 9239.40820, #n: 20 20)
GFI1_L400F_R1_8mers.txt 0.23297753143389824 78964.95 83128.266 1 	 (std: 12648.97168 13194.69922, #n: 20 20)
HOXC4_N178S_R1_8mer

In [ ]:
cs.uniform_baseline(
    3,
    tf_labels,
    20, 
    filenames, 
    ack_regret_fn, 
    #ack_rel_opt_val_data_extractor,
    arrs,
    to_eval[5], 
    14, 
    pval_threshold=1, 
)

In [ ]:
cs.prop_test(100, 
             filenames, 
             idx_data_extractor, 
             arrs, 
             [to_eval[-2], to_eval[-1]], 
             9,
             pval_threshold=0.2,
             paired_test=True,
            )

In [ ]:
cs.prop_test(100, filenames, ir_regret_fn, arrs, [to_eval[-2], to_eval[-1]], 9, pval_threshold=0.2, paired_test=True)

In [ ]:
cs.prop_test(100, filenames, ack_regret_fn, arrs, [to_eval[-2], to_eval[-1]], 9, pval_threshold=1.4, paired_test=True)

In [ ]:
cs.prop_test(200, 
             filenames, 
             idx_data_extractor, 
             arrs, 
             [to_eval[-2], to_eval[-1]], 
             29, 
             pval_threshold=1,
             paired_test=True,
            )

In [ ]:
cs.prop_test(200, filenames, ack_rel_opt_val_data_extractor, arrs, [to_eval[-2], to_eval[-1]], 29, pval_threshold=1.4, paired_test=True)

In [62]:
to_eval = [
    'ensemble14/o_none_ucb_',
    'ensemble14/o_none_ucb_maxinvar_g000510204080_',
    'ensemble14/o_none_ucb_maxvar_g000510204080_',
    'ensemble14/o_none_ucb_adve_',
    'ensemble14/o_none_ucb_nc_g000510204080_',
    #'ensemble14/o_none_ucb_mincov_g000510204080_',
]

In [65]:
ack_iter = 29
for batch_size in [10]:
    print("BATCH SIZE", batch_size)
    for filename in filenames:
        filename_printed = False
        for experiment in to_eval:
            if len(arrs[experiment][0]) == 0:
                continue
            idx_frac = None
            num_used = 0
            val_nll = 0
            try:
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) <= ack_iter:
                        continue
                    num_used += 1
                    mean = 0
                    #for i in range(len(stats[filename][batch_size])):
                        #mean += stats[filename][batch_size][i]['logging'][1]['best']['nll']
                    if len(stats[filename][batch_size]) > 0:
                        mean /= (len(stats[filename][batch_size]))
                    #val_nll += stats[filename][batch_size][0]['logging'][1]['best']['nll']
                    val_nll += mean
                    if idx_frac is None:
                        idx_frac = stats[filename][batch_size][ack_iter]['idx_frac']
                        #idx_frac = tf_labels[filename][stats[filename][batch_size][ack_iter]['ir_batch_cur_idx'][-1]]
                        #idx_frac = stats[filename][batch_size][ack_iter]['corr_stats'][1]
                    else:
                        idx_frac = idx_frac + stats[filename][batch_size][ack_iter]['idx_frac']
                        #idx_frac = torch.max(idx_frac, stats[filename][batch_size][ack_iter]['idx_frac'])
                        #idx_frac = idx_frac + tf_labels[filename][stats[filename][batch_size][ack_iter]['ir_batch_cur_idx'][-1]]
                        #idx_frac = idx_frac + stats[filename][batch_size][ack_iter]['corr_stats'][1]
                if num_used > 0:
                    if not filename_printed:
                        print(filename)
                        filename_printed = True
                    idx_frac = idx_frac/num_used
                    #idx_frac = idx_frac.numpy()
                    print(arrs[experiment][1] + ":", " ".join(("{:6.6f}".format(k) for k in idx_frac)), ";", "{:6.2f}".format(val_nll/num_used), "val_nll ;", str(num_used), "samples")
                    #print(arrs[experiment][1] + ":", str(idx_frac), ";", "{:6.2f}".format(val_nll/num_used), "val_nll ;", str(num_used), "samples")
            except Exception as e:
                pass

BATCH SIZE 10
HESX1_E149K_R1_8mers.txt
normal: 0.393140 0.146472 0.080921 0.042552 ;   0.00 val_nll ; 20 samples
maxvar_g80: 0.422256 0.146290 0.079735 0.042180 ;   0.00 val_nll ; 20 samples
adve: 0.403963 0.147536 0.080952 0.042499 ;   0.00 val_nll ; 20 samples
nc: 0.395884 0.147901 0.081073 0.042529 ;   0.00 val_nll ; 20 samples
EGR2_D383Y_R1_8mers.txt
normal: 0.120427 0.058516 0.039739 0.027443 ;   0.00 val_nll ; 20 samples
maxvar_g80: 0.142988 0.069799 0.044436 0.028842 ;   0.00 val_nll ; 20 samples
adve: 0.135061 0.065389 0.042855 0.028500 ;   0.00 val_nll ; 20 samples
nc: 0.120122 0.061283 0.040742 0.028127 ;   0.00 val_nll ; 20 samples
BCL6_H676Y_R1_8mers.txt
normal: 0.397409 0.156873 0.086014 0.043753 ;   0.00 val_nll ; 20 samples
maxvar_g80: 0.391616 0.155596 0.085315 0.043646 ;   0.00 val_nll ; 20 samples
adve: 0.396951 0.156144 0.085771 0.043722 ;   0.00 val_nll ; 20 samples
nc: 0.400153 0.157786 0.085877 0.043768 ;   0.00 val_nll ; 20 samples
ARX_L343Q_R1_8mers.txt
normal: 

In [ ]:
import collect_stats as cs

In [ ]:
cs.plot_data_vs_ack_iter(
    20,
    filenames,
    'avg_seeds',
    'relative optimal value',
    #ack_regret_fn,
    ack_rel_opt_val_data_extractor,
    14,
    arrs,
    to_eval,
    legend_loc=3,
    figsize=(6, 4),
    num_samples_label=True,
    save_path='/cluster/sj1/bb_opt/plots',
)

In [ ]:
cs.plot_data_vs_ack_iter(
    10,
    filenames[0:15],
    'avg_seeds',
    'gamma',
    lambda x : x['best_gamma'] if x['best_gamma'] is not None else 0,
    30,
    arrs,
    to_eval,
    legend_loc=3,
)

In [ ]:
ack_iter = 0
for filename in filenames:
    print(filename)
    for batch_size in batches:
        for experiment in to_eval:
            best_value = 0
            num_used = 0
            for stats in arrs[experiment][0]:
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                num_used += 1
                best_value += stats[filename][batch_size][ack_iter]['logging'][9].item()
            if num_used > 0:
                best_value = best_value/num_used
                print(arrs[experiment][1] + ":", best_value, ";", str(num_used), "samples")

In [ ]:
test_properties = [
    'test_log_prob',
    'test_mse',
    'test_kt_corr',
    'test_std_list',
    'test_mse_std_corr'
]

avg = True

for prop in test_properties[1:2]:
    for filename in filenames[0:1]:
        print(filename)
        for batch_size in batches:
            for experiment in arrs:
                plt.figure(figsize=(15, 4))
                plt.subplot(121)
                prop_list = np.array([0.]*num_acks)
                num_used = 0
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) < num_acks:
                        continue
                    num_used += 1
                    if avg:
                        prop_list += np.array([stats[filename][batch_size][i][prop][0].item() for i in range(num_acks)])
                    else:
                        prop_list = [stats[filename][batch_size][i][prop][0].item() for i in range(num_acks)]
                    if not avg:
                        plt.plot(prop_list)
                if avg:
                    plt.plot(prop_list/num_used)
                plt.title(arrs[experiment][1] + "; " + prop + "; all")
                plt.subplot(122)
                prop_list = np.array([0.]*num_acks)
                num_used = 0
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) < num_acks:
                        continue
                    num_used += 1
                    if avg:
                        prop_list += np.array([stats[filename][batch_size][i][prop][1].item() for i in range(num_acks)])
                    else:
                        prop_list = [stats[filename][batch_size][i][prop][1].item() for i in range(num_acks)]
                    if not avg:
                        plt.plot(prop_list)
                if avg:
                    plt.plot(prop_list/num_used)
                plt.title(arrs[experiment][1] + "; " + prop + "; top 10%")

In [ ]:
ack_iter = 0
for filename in filenames[2:3]:
    print(filename)
    for batch_size in batches:
        for experiment in arrs:
            plt.figure(figsize=(15, 4))
            for stats in arrs[experiment][0]:
                #for i in range(num_acks):
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                prop_list = stats[filename][batch_size][ack_iter]['logging'][-1].numpy()
                plt.plot(prop_list)
                plt.title(arrs[experiment][1] + "; " + str(ack_iter))

In [ ]:
to_eval = [
    #'top_val_none_ucb_g0.0_',
    'top_val_none_ucb_maxstd_g01051020_',
    'top_val_none_ucb_g0.0_200epochs_last_',
    #'top_val_none_ucb_maxvar_g01051020_',
]

In [ ]:
ack_iter = 1
for filename in filenames[:3]:
    print(filename)
    for batch_size in batches:
        for experiment in to_eval:
            plt.figure(figsize=(15, 4))
            for stats in arrs[experiment][0]:
                #for i in range(num_acks):
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                #prop_list = stats[filename][batch_size][ack_iter]['logging'][5].numpy()
                prop_list = stats[filename][batch_size][ack_iter]['best_gamma']
                plt.plot(prop_list)
                plt.title(arrs[experiment][1] + "; " + str(ack_iter))

In [ ]:
test_properties = [
    'test_log_prob',
    'test_mse',
    'test_kt_corr',
    'test_std_list',
    'test_mse_std_corr'
]

ack_iter = -1
for filename in filenames:
    print(filename)
    for batch_size in [10]:
        for experiment in to_eval:
            m = None
            num_used = 0
            for stats in arrs[experiment][0]:
                #for i in range(num_acks):
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                num_used += 1
                #m += stats[filename][batch_size][ack_iter]['test_std_list'][0].item()
                m2 = np.array([
                    np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]),
                    #stats[filename][batch_size][ack_iter]['test_std_list'][0].item(),
                    #stats[filename][batch_size][ack_iter]['test_std_list'][1].item(),
                    #stats[filename][batch_size][ack_iter]['test_std_list'][2].item()
                ])
                if m is None:
                    m = m2
                else:
                    m += m2
            if num_used > 0:
                print(arrs[experiment][1], ":", m/num_used, ";", num_used, "samples")

In [ ]:
num_labels = 32897

In [ ]:
to_eval = [
    'take_log_none_ucb_c1.0_g0.0_',
    #'take_log_none_ucb_c2.0_g0.0_',
    'take_log_none_ucb_c1.0_g5.0_',
    'take_log_none_ucb_c1.0_g10.0_',
    'take_log_none_ucb_c1.0_g20.0_',
]

to_eval = [
    'take_log_none_ucb_num_acks_30_c1.0_g0.0_',
    'take_log_none_ucb_num_acks_30_c1.0_g5.0_',
    'take_log_none_ucb_num_acks_30_c1.0_g10.0_',
]

In [ ]:
num_acks = 30

In [ ]:
to_eval = {
    'take_log_none_ucb_num_acks_30_c1.0_g0.0_': 'Normal',
    'take_log_none_ucb_num_acks_30_c1.0_maxvar_' : 'MOD',
    'take_log_none_ucb_num_acks_30_c1.0_mincorr_' : 'MOD2',
}

to_eval = {
    'take_log_none_ucb_ntest' : 'Normal',
}

In [ ]:
to_eval = [
    #'top_val_none_ucb_g0.0_',
    'top_val_none_ucb_g0.0_modelseed',
    #'top_val_none_ucb_maxstd_g01051020_',
    #'top_val_none_ucb_g0.0_200epochs_last_',
    #'top_val_none_ucb_maxvar_g01051020_',
]

In [ ]:
avg = True
num_acks = 10

for batch_size in batches:
    for filename in filenames:
        if avg:
            plt.figure(figsize=(6, 4))
            legend = []
        for experiment in to_eval:
            if not avg:
                plt.figure(figsize=(15, 4))
            prop_list = []
            num_used = 0
            for stats in arrs[experiment][0]:
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                if len(stats[filename][batch_size]) < num_acks:
                    continue
                num_used += 1
                if avg:
                    #prop_list += [np.array([np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]) for ack_iter in range(num_acks)])]
                    #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['idx_frac'][0]) for ack_iter in range(num_acks)])]
                    #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][0][-4]) for ack_iter in range(num_acks)])]
                    prop_list += [np.array([(stats[filename][batch_size][ack_iter]['logging'][0][8]) for ack_iter in range(num_acks)])]
                else:
                    prop_list = [(stats[filename][batch_size][ack_iter]['ack_labels'].max()) for ack_iter in range(num_acks)]
                    prop_list = [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][0][-4]) for ack_iter in range(num_acks)])]
                if not avg:
                    plt.plot(prop_list)
            if avg and num_used > 0:
                prop_list = np.stack(prop_list, axis=0)
                plt.plot(np.median(prop_list, axis=0))
                legend += [arrs[experiment][1]]
            elif not avg:
                plt.title(arrs[experiment][1] + "; " + str(batch_size) + "; all")
        if avg:
            plt.legend(legend)
            plt.xlabel('Acquisition Iteration')
            plt.ylabel('Relative optimal value')
            #plt.title(filename + "; " + str(batch_size) + "; all")
            plt.title(filename.split("_")[0])

In [ ]:
avg = True

for batch_size in batches:
    for filename in filenames:
        if avg:
            plt.figure(figsize=(6, 4))
            legend = []
        for experiment in to_eval:
            if not avg:
                plt.figure(figsize=(15, 4))
                
            for prop_i in [5, 6]:
                prop_list = []
                num_used = 0
                for stats in arrs[experiment][0]:
                    if filename not in stats:
                        continue
                    if batch_size not in stats[filename]:
                        continue
                    if len(stats[filename][batch_size]) < num_acks:
                        continue
                    num_used += 1
                    if avg:
                        #prop_list += [np.array([np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]) for ack_iter in range(num_acks)])]
                        #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['idx_frac'][0]) for ack_iter in range(num_acks)])]
                        #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][1][prop_i].item()) for ack_iter in range(num_acks)])]
                        prop_list += [np.array([(stats[filename][batch_size][ack_iter]['best_gamma']) for ack_iter in range(num_acks)])]
                    else:
                        prop_list = [(stats[filename][batch_size][ack_iter]['ack_labels'].max()) for ack_iter in range(num_acks)]
                        prop_list = [np.array([(stats[filename][batch_size][ack_iter]['corr_stats'][1][prop_i].item()) for ack_iter in range(num_acks)])]
                    if not avg:
                        plt.plot(prop_list)
                if avg and num_used > 0:
                    prop_list = np.stack(prop_list, axis=0)
                    plt.plot(np.median(prop_list, axis=0))
                    legend += [str(prop_i)]
                elif not avg:
                    plt.title(arrs[experiment][1] + "; " + str(batch_size) + "; all")
        if avg:
            plt.legend(legend)
            plt.xlabel('Acquisition Iteration')
            plt.ylabel('Relative optimal value')
            #plt.title(filename + "; " + str(batch_size) + "; all")
            plt.title(filename.split("_")[0])

In [ ]:
avg = True

plt.figure(figsize=(6, 4))
legend = []
for experiment in to_eval:
    prop_list = []
    for batch_size in [10]:
        for filename in filenames:
            for stats in arrs[experiment][0]:
                if filename not in stats:
                    continue
                if batch_size not in stats[filename]:
                    continue
                if len(stats[filename][batch_size]) < num_acks:
                    continue
                prop_list += [np.array([np.exp(stats[filename][batch_size][ack_iter]['ack_labels'].max()-tf_max[filename]) for ack_iter in range(num_acks)])]
                #prop_list += [np.array([(stats[filename][batch_size][ack_iter]['idx_frac'][0]) for ack_iter in range(num_acks)])]
    prop_list = np.stack(prop_list, axis=0)
    plt.plot(np.median(prop_list, axis=0))
    #legend += [arrs[experiment][1]]
    legend += [to_eval[experiment]]
plt.legend(legend)
plt.xlabel('Acquisition Iteration')
plt.ylabel('Relative optimal value')